In [ ]:
import rustxes
import json

In [ ]:
ocel_xml_path = "/home/aarkue/doc/projects/rust-bridge-process-mining/process_mining/src/event_log/tests/test_data/order-management.xml"
ocel_rs = rustxes.import_ocel_xml_pm4py(ocel_xml_path)

In [ ]:
import pm4py

ocel_py = pm4py.read_ocel2_xml(ocel_xml_path)

In [ ]:
assert ocel_rs.events.equals(ocel_py.events.convert_dtypes().reset_index(drop=True))
assert ocel_rs.objects.equals(ocel_py.objects.convert_dtypes().reset_index(drop=True))
assert ocel_rs.relations.equals(ocel_py.relations.convert_dtypes().reset_index(drop=True))
assert ocel_rs.o2o.equals(ocel_py.o2o.convert_dtypes().reset_index(drop=True))
assert ocel_rs.get_summary() == ocel_py.get_summary()


# This will still fail! (see below)
assert ocel_rs.object_changes.equals(ocel_py.object_changes.convert_dtypes().reset_index(drop=True))

In [ ]:
ocel_json_path = "/home/aarkue/doc/projects/rust-bridge-process-mining/process_mining/src/event_log/tests/test_data/order-management.json"
ocel_json_rs = rustxes.import_ocel_json_pm4py(ocel_json_path)

In [ ]:
import pm4py

ocel_json_py = pm4py.read_ocel2_json(ocel_json_path)

In [ ]:
def sort_df_columns(df):
  return df.reindex(sorted(df.columns), axis=1)

def equals_with_sorted_columns(df1,df2):
  df1 = sort_df_columns(df1)
  df2 = sort_df_columns(df2)
  if not df1.equals(df2):
    from pandas.testing import assert_frame_equal
    try:
      assert_frame_equal(df1,df2)
    except Exception as e:
      print(e)
    finally:
      return False
  else:
    return True

In [ ]:
print("\nEvents:\n" + str(equals_with_sorted_columns(ocel_json_rs.events,ocel_json_py.events.convert_dtypes().reset_index(drop=True))))
print("\nObjects:\n" + str(equals_with_sorted_columns(ocel_json_rs.objects,ocel_json_py.objects.convert_dtypes().reset_index(drop=True))))
print("\nRelations:\n" + str(equals_with_sorted_columns(ocel_json_rs.relations,ocel_json_py.relations.convert_dtypes().reset_index(drop=True))))
print("\nO2O:\n" + str(equals_with_sorted_columns(ocel_json_rs.o2o,ocel_json_py.o2o.convert_dtypes().reset_index(drop=True))))
print("\nSummary:\n" + str(ocel_json_rs.get_summary() == ocel_json_py.get_summary()))


# This will still fail! (see below)
print("\nObject Changes:\n" + str(equals_with_sorted_columns(ocel_json_rs.object_changes,ocel_json_py.object_changes.convert_dtypes().reset_index(drop=True))))

In [ ]:
import pm4py

ocel_pm4py = pm4py.ocel.OCEL(
    events=ocel_rs["events"].to_pandas().convert_dtypes(),
    objects=ocel_rs["objects"].to_pandas().convert_dtypes(),
    relations=ocel_rs["relations"].to_pandas().convert_dtypes(),
    globals={},
    o2o=ocel_rs["o2o"].to_pandas().convert_dtypes(),
    parameters={
        pm4py.objects.ocel.obj.Parameters.EVENT_ID: "ocel:eid",
        pm4py.objects.ocel.obj.Parameters.EVENT_ACTIVITY: "ocel:activity",
        pm4py.objects.ocel.obj.Parameters.EVENT_TIMESTAMP: "ocel:timestamp",
        pm4py.objects.ocel.obj.Parameters.OBJECT_ID: "ocel:oid",
        pm4py.objects.ocel.obj.Parameters.OBJECT_TYPE: "ocel:type",
        pm4py.objects.ocel.obj.Parameters.QUALIFIER: "ocel:qualifier",
        pm4py.objects.ocel.obj.Parameters.CHANGED_FIELD: "ocel:field",
    },
)

In [ ]:
ocel_py.get_summary() == ocel_pm4py.get_summary()

In [ ]:
ocel_rs.keys()

In [ ]:
o2o_rs = ocel_rs["o2o"].to_pandas().convert_dtypes()
o2o_rs.equals(ocel_py.o2o.convert_dtypes())

In [ ]:
objects_rs = ocel_rs["objects"].to_pandas().convert_dtypes()
objects_rs.equals(ocel_py.objects.convert_dtypes())

In [ ]:
events_rs = ocel_rs["events"].to_pandas().convert_dtypes()
events_rs.equals(ocel_py.events.convert_dtypes().reset_index(drop=True))

In [ ]:
relations_rs = ocel_rs["relations"].to_pandas().convert_dtypes()
relations_rs.equals(ocel_py.relations.convert_dtypes().reset_index(drop=True))

In [ ]:
from pandas.testing import assert_frame_equal
object_changes_rs = sort_df_columns(ocel_rs["object_changes"].to_pandas().convert_dtypes())
object_changes_py = sort_df_columns(ocel_py.object_changes.convert_dtypes().reset_index(drop=False))
object_changes_py["ocel:timestamp"] = object_changes_py["ocel:timestamp"].astype("datetime64[ns, UTC]")
object_changes_py = object_changes_py.sort_values(["ocel:timestamp","index"]).drop("index",axis=1).reset_index(drop=True)
# Note: Timestamps don't match!
# I think the Rust implementation parses them correctly (Wed May 22 2024 01:00:00 GMT+0200 (Mitteleuropäische Sommerzeit) => 2024-05-21T23:00:00.000Z),
# BUT: this is a very weird format and should be fixed upstream in the OCEL logs
# Also: pm4py implementation currently doesn't sort wrt. to timestamp for object changes (but it does for events, ...)
print(object_changes_rs.drop("ocel:timestamp",axis=1).equals(object_changes_py.drop("ocel:timestamp", axis=1)))
if not object_changes_rs.equals(object_changes_py): 
  assert_frame_equal(object_changes_rs,object_changes_py)


In [ ]:
set1 = set(ocel_rs["objects"]["ocel:oid"].unique())
set2 = set(ocel_py.objects["ocel:oid"].unique())
print("Is rust object set a superset of py? " + str(set1.issuperset(set2)))
res = set1 - set2
res

In [ ]:
"material:983" in set2

In [ ]:
import pandas as pd
from pandas import DataFrame

# Rough comparison checks of two DataFrames
# Ignores certain differences (e.g., different datatypes) on purpose
def compare_log_dfs(df1: DataFrame,df2: DataFrame):
  col_diff =  set(df1.columns).symmetric_difference(set(df2.columns));
  if len(col_diff) > 0:
      print(f"❌ Column are different: {col_diff}")
      return False

  all_columns = set(df1.columns).union(set(df2.columns))
  all_equal = True
  for col in all_columns:
    # or all(...) to handle some weird datatype stuff; if all values are equal, we also consider the column as equal
    if  df1[col].equals(df2[col]) or all([v1 == v2 or (pd.isnull(v1) and pd.isnull(v2)) or str(v1) == str(v2) for (v1,v2) in zip(df1[col],df2[col])]):
      pass
    else:
      all_equal = False
      print(f"❌ For column {col} not all entries are equal!")
  if all_equal:
    print(f"✅✅✅ All values in all column are equal!")
    return True
  else:
    print(f"❌❌❌ NOT all values in all column are equal!")
    return False

In [ ]:
log_import_res = dict()

In [ ]:
import time
import pm4py
import numpy as np
import glob
import pandas as pd

for log_name in glob.glob("/home/aarkue/dow/event_data/BPI_Challenge_2017*.xes"):
  print(log_name)
  log_name_postfix = log_name.split("/")[-1]

  # Measure execution times
  start = time.time()
  (log_rs,other_data_str) = rustxes.import_xes(log_name)
  log_rs = log_rs.to_pandas() if log_rs.shape != (0,0) else pd.DataFrame()
  log_rs = log_rs.fillna(np.nan)
  rs_dur = time.time() - start
  start = time.time()
  log_py_lbl = pm4py.read_xes(log_name,variant="line_by_line")
  py_lbl_dur = time.time() - start
  start = time.time()
  try:
    log_py_iter = pm4py.read_xes(log_name,variant="iterparse")
  except Exception as e:
    print(e)
  py_iter_dur = time.time() - start

  # Print & save execution times in log_import_res for plotting
  print(rs_dur,py_lbl_dur,py_iter_dur)
  log_import_res[log_name_postfix] = {"Rust": rs_dur, "PM4Py (line_by_line)": py_lbl_dur, "PM4Py (iterparse)": py_iter_dur }
  # Check if DataFrames are _roughtly_ equal
  if not compare_log_dfs(log_rs,log_py_iter):
    print("WARNING: Logs do not match for " + log_name)

In [ ]:
import pandas as pd
df = pd.DataFrame(log_import_res)
import plotly.express as px
fig = px.bar(df.transpose(), barmode="group", labels={"index": "Event Log", "value": "Parse Duration [s]", "variable": "Parsing Implementation"})
fig.show()
for log_name in log_import_res: 
  x = [ "Rust", "PM4Py (line_by_line)","PM4Py (iterparse)"]
  fig = px.bar(labels={"x": "XES Parser", "y": "Parse Duration [s]"}, title=f"XES Parsing Performance on <i>{log_name}</i>",x=x, y=[log_import_res[log_name][n] for n in x],color=x)
  fig.show()